In [ ]:
import os

In [ ]:
import pandas as pd

In [ ]:
METHODS = ['default', 'ror','uniform','nri_uniform','var']
DATA_LBP_PATH = 'dataset/fish_data_lbp/resize/'

In [ ]:
METHOD_INDEX = 4
df = pd.read_csv(os.path.join(DATA_LBP_PATH,METHODS[METHOD_INDEX],METHODS[METHOD_INDEX]+'_dense_radius_is_3.csv'), header=None)
df.head(20)

In [ ]:
from sklearn import preprocessing
lb = preprocessing.LabelEncoder()
lb.fit(df[1].unique())

In [ ]:
new_class = lb.transform(df[1])
new_class
# df.head()


In [ ]:
# from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [ ]:
index_input =[i for i in range(4,260)]
X_train, X_test, y_train, y_test = train_test_split(df[index_input].copy().to_numpy(), new_class, 
                                                    test_size=0.333,random_state=1)

In [ ]:
y_test

In [ ]:
import torch as tr
import torch.nn as tr_nn
import torch.nn.functional as tr_fn
import torch.optim as tr_optim
# import torchvision
# import torchvision.transforms as tr_transforms


In [ ]:
device = 'cpu'
if(tr.cuda.is_available()):
    device = tr.device('cuda')    
print(device)


In [ ]:
# x_test_tr = tr.from_numpy(X_test).view(X_test.shape[0],256,1).type(tr.float).to(device)
# x_train_tr = tr.from_numpy(X_train).view(X_train.shape[0],256,1).type(tr.float).to(device)
# y_test_tr = tr.from_numpy(y_test).view(-1,1).type(tr.float).to(device)
# y_train_tr = tr.from_numpy(y_train).view(-1,1).type(tr.float).to(device)
# print(x_train_tr.shape, y_train_tr)

x_test_tr = tr.randn(X_test.shape[0], 20)
x_train_tr = tr.randn(X_train.shape[0], 20)
y_test_tr = tr.from_numpy(y_test).view(-1,1).type(tr.float)
y_train_tr = tr.from_numpy(y_train).view(-1,1).type(tr.float)
print(x_train_tr.shape)

In [ ]:
import math
batch_size = 300
rows_size = X_train.shape[0]
batch_rows = rows_size/batch_size
print(batch_rows)
batch_epoch = math.ceil(batch_rows)
print(batch_epoch)

In [ ]:
class Net(tr_nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = tr_nn.Linear(20, 500)
        self.pool = tr_nn.MaxPool1d(1, 2)
        self.conv2 = tr_nn.Linear(500, 326)
        self.fc1 = tr_nn.Linear(326, 120)
        self.fc2 = tr_nn.Linear(120, 84)
        self.fc3 = tr_nn.Linear(84, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # x = tr.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x


In [ ]:
model = Net().to(device)
model

In [ ]:

# criterion = tr_nn.BCELoss()
# optimizer = tr_optim.Adam(model.parameters(), lr=0.001)
criterion = tr_nn.CrossEntropyLoss()
optimizer = tr_optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:


for epoch in range(2000):  # loop over the dataset multiple times
    running_loss = 0.0
    for i in range(0,batch_epoch):
        # get the inputs; data is a list of [inputs, labels]
        endp = (i+1)*batch_size
        inputs = x_train_tr[i*batch_size:endp if endp < rows_size else rows_size,:]
        labels = y_train_tr [i*batch_size:endp if endp < rows_size else rows_size]
        labels = labels.to(device)
        inputs = inputs.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # # print statistics
        running_loss += loss.item()
        if epoch % 10 == 0:    # print every 2000 mini-batches
            print(outputs[0],labels[0])
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss:.3f}')
        running_loss = 0.0
    
    
    
print('Finished Training')

In [ ]:
with tr.no_grad():
    y_pr = model(x_test_tr)
    y_pr_class = y_pr.round()
    acc = y_pr_class.eq(y_test_tr).sum()/float(y_test_tr.shape[0])
    # for i in range(0,y_pr_class.shape[0]):
    #     print(y_pr_class[i],y_test_tr[i])
    print("acc", acc)

In [ ]:
conv1 = tr_nn.Conv1d(256, 200, 1)
pool = tr_nn.MaxPool1d(1)
conv2 = tr_nn.Conv1d(200, 16, 1)
fc1 = tr_nn.Linear(16, 120)
fc2 = tr_nn.Linear(120, 84)
fc3 = tr_nn.Linear(84, 10)

def tt(x):
    x = pool(tr_fn.relu(conv1(x)))
    x = pool(tr_fn.relu(conv2(x)))
    x = tr.flatten(x, 1) # flatten all dimensions except batch
    x = tr_fn.relu(fc1(x))
    x = tr_fn.relu(fc2(x))
    x = fc3(x)
    return x
# input = tr.randn(20, 15, 50)
# print(input[2].shape)
# print(m)
yyy = x_test_tr[i*batch_size:(i+1)*batch_size,:]
output = tt(yyy)
output